In [ ]:
# Setup and install necessary libraries
!pip install langchain_community sentence-transformers pymupdf chromadb transformers faiss-cpu PyPDF2 pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 607.0/607.0 kB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 69.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 22.2 MB/s et

In [ ]:
# Mount Google Drive (if using in Colab)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Import necessary libraries
import fitz
import pdfplumber
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
# Function to load PDF text using pymupdf
def load_pdf_with_pymupdf(file_path):
    try:
        text = ""
        with fitz.open(file_path) as pdf:
            for page in pdf:
                text += page.get_text()
        text = ' '.join(text.split())  # Clean up whitespace
        return text
    except Exception as e:
        print(f"Failed to load PDF using pymupdf: {e}")
        return ""

# Function to load PDF text using pdfplumber
def load_pdf_with_pdfplumber(file_path):
    try:
        text = ""
        with pdfplumber.open(file_path) as pdf:
            for page in pdf.pages:
                text += page.extract_text() if page.extract_text() else ""
        text = ' '.join(text.split())  # Clean up whitespace
        return text
    except Exception as e:
        print(f"Failed to load PDF using pdfplumber: {e}")
        return ""

In [ ]:
# Choose one of the functions to load PDF text
#file_path = "/content/drive/MyDrive/Colab Notebooks/IETM_sLM_ARG/Artificial Intelligence Industry Trend Brief.pdf"  # Example file path
file_path = "/content/drive/MyDrive/Colab Notebooks/IETM_sLM_ARG/data.pdf"  # Example file path
pdf_text_pymupdf = load_pdf_with_pymupdf(file_path)
pdf_text_pdfplumber = load_pdf_with_pdfplumber(file_path)

In [ ]:
# 두 방법의 출력을 비교
if pdf_text_pymupdf:
    print("Loaded text using pymupdf:", pdf_text_pymupdf[:500])
else:
    print("Loaded text using pdfplumber:", pdf_text_pdfplumber[:500])


Loaded text using pymupdf: KOREA AEROSPACE INDUSTRIES, LTD ROKAF SERIES AIRCRAFT TA-50 무장발사 비행교범 지휘관은대상항공기의작동에관련된공군관계자에게이발행물을알릴책임이있다. K.T.O. 1T-50C-34-1-1 2011. 10. 31 Rev. A 변경판3 2012. 12. 31 K.T.O. 1T-50C-34-1-1 T-2 배 포 본기술도서는대한민국공군의배포인가처에한해배포가능하며, 군사자료이므로허용된이외의목적으로 사용할경우, 처벌을받을수있습니다. 또한, 본기술도서는국방부규정에의한기밀등급을유지해야하며공 군의인가없이는등급을낮추거나기밀해제조처할수없습니다. 본기술도서에대한요구사항은대구광역시 동구검사동사서함304-101호공군군수사령부정비부중앙기술도서관리소로연락바랍니다. 보충판주의 본기술도서는 K.T.0. 1T-50C-34-1-2가없이는완전하지않다. 대체주의 비행교범, 보충판및점검표현황은교범목록K.T.O. 1T-50A-01을참조한다. K.T.O. 1T-50C-34-1-1 A 변


In [ ]:
# 로드된 텍스트를 추가 처리를 위해 청크로 분할합니다.
def split_text(text, chunk_size=1000, chunk_overlap=100):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\\n\\n", "\\n", " ", ""]
    )
    split_texts = splitter.split_text(text)
    return split_texts


In [ ]:
# 추출된 텍스트 사용 (필요에 따라 조정)
text_chunks = split_text(pdf_text_pymupdf if pdf_text_pymupdf else pdf_text_pdfplumber)
print(f"Number of chunks: {len(text_chunks)}")
print(text_chunks[:3])

Number of chunks: 379
['KOREA AEROSPACE INDUSTRIES, LTD ROKAF SERIES AIRCRAFT TA-50 무장발사 비행교범 지휘관은대상항공기의작동에관련된공군관계자에게이발행물을알릴책임이있다. K.T.O. 1T-50C-34-1-1 2011. 10. 31 Rev. A 변경판3 2012. 12. 31 K.T.O. 1T-50C-34-1-1 T-2 배 포 본기술도서는대한민국공군의배포인가처에한해배포가능하며, 군사자료이므로허용된이외의목적으로 사용할경우, 처벌을받을수있습니다. 또한, 본기술도서는국방부규정에의한기밀등급을유지해야하며공 군의인가없이는등급을낮추거나기밀해제조처할수없습니다. 본기술도서에대한요구사항은대구광역시 동구검사동사서함304-101호공군군수사령부정비부중앙기술도서관리소로연락바랍니다. 보충판주의 본기술도서는 K.T.0. 1T-50C-34-1-2가없이는완전하지않다. 대체주의 비행교범, 보충판및점검표현황은교범목록K.T.O. 1T-50A-01을참조한다. K.T.O. 1T-50C-34-1-1 A 변경3 최신 변경판을삽입하라; 대체판은해당규정에따라파기하라. 주 기: 변경판에의해변경된부분은그페이지바깥쪽여백에수직선으로표시된다. 원판및변경판발행일: 본기술도서는총332페이지이며, 다음과같이구성되어있다. 페이지 *변경판 페이지 *변경판 번호 번호 번호 번호 *이난의“0”은원판을의미함. 유효페이지목록 원판........... 0 .......... 11.10.31 변경판....... 1 .......... 12.04.30 변경판....... 2 .......... 12.10.15 변경판....... 3 .......... 12.12.31 Title ------------------------------------------------- 3 T-2 -------------------------------------------------- 0 A ---------------------------------------------------- 3 i',

In [ ]:
# SentenceTransformer를 사용하여 임베딩 생성
embedding_model = SentenceTransformer('jhgan/ko-sroberta-multitask')
def embed_text_chunks(text_chunks):
    embeddings = embedding_model.encode(text_chunks, convert_to_tensor=True, show_progress_bar=True)
    return embeddings

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.86k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# 텍스트 청크에 대한 임베딩 생성
text_embeddings = embed_text_chunks(text_chunks)
print(f"Number of embeddings: {len(text_embeddings)}")
print(text_embeddings[0])  # Example output of the first embedding

Batches:   0%|          | 0/12 [00:00<?, ?it/s]

Number of embeddings: 379
tensor([ 5.7133e-03,  1.0495e-01, -2.0921e-01,  1.1020e-01,  5.0559e-01,
        -8.5469e-03, -4.3656e-01,  5.7418e-01, -1.1965e-01, -7.5558e-02,
         3.2098e-01, -9.7226e-02, -4.5578e-01,  6.5706e-01, -1.6963e-01,
         5.6444e-01, -2.3060e-01,  3.3747e-01,  4.1635e-01, -8.3433e-02,
        -3.5542e-01,  3.6421e-01,  3.6972e-01, -3.1260e-01,  3.2564e-01,
         1.9605e-01,  1.6250e-01, -8.9423e-02,  3.2631e-01,  2.0921e-02,
         7.4829e-02, -3.2577e-01,  1.3371e-01,  4.0066e-01,  2.9282e-01,
        -7.3775e-02, -2.4282e-01,  1.3589e-01, -1.0866e-01,  7.0558e-02,
        -6.5831e-01,  3.2234e-01,  1.1965e-03,  2.9471e-01,  3.8740e-01,
        -1.5256e-01, -4.6077e-02, -3.0216e-01, -6.4044e-01,  1.8185e-01,
         1.8482e-02, -9.2833e-02,  2.4875e-01,  6.8555e-01, -2.2674e-01,
         9.2301e-03,  4.5014e-01, -2.1643e-01,  1.2312e-01,  7.1432e-02,
        -5.8507e-01, -2.8056e-02, -4.0202e-01,  1.9795e-01, -2.4625e-01,
        -1.0544e-01, -3.8

In [ ]:
import chromadb
from chromadb.config import Settings

# ChromaDB 설정 및 클라이언트 초기화 (새로운 방식)
chroma_client = chromadb.Client(Settings(
    persist_directory="/content/drive/MyDrive/Colab Notebooks/chroma_db" # 데이터 저장 경로 설정
))

# 컬렉션 생성
#collection_name = "ai_industry_trends"
collection_name = "Flight_Manual"
if collection_name in chroma_client.list_collections():
    collection = chroma_client.get_collection(collection_name)
else:
    collection = chroma_client.create_collection(name=collection_name)

# 텍스트 조각을 벡터 임베딩과 함께 컬렉션에 추가하는 함수
def store_embeddings_in_chroma(text_chunks, embeddings):
    # 문서 ID를 생성
    ids = [f"doc_{i}" for i in range(len(text_chunks))]

    # 컬렉션에 데이터 추가 (documents 필드 추가)
    collection.add(
        ids=ids,
        embeddings=[embedding.tolist() for embedding in embeddings],  # 벡터를 리스트로 변환
        metadatas=[{"text": chunk} for chunk in text_chunks],           # 메타데이터에 원본 텍스트 저장
        documents=text_chunks                                           # documents 필드에 원본 텍스트 추가
    )
    print(f"Added {len(text_chunks)} documents to the ChromaDB collection.")


# 임베딩을 ChromaDB에 저장
store_embeddings_in_chroma(text_chunks, text_embeddings)


Added 379 documents to the ChromaDB collection.


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings

# 허깅페이스 모델과 토크나이저 로드
model_name = "KISTI-KONI/KONI-Llama3-8B-Instruct-20240729"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# GPU 메모리 최적화를 위한 설정 (float16 + device_map)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"  # 자동 메모리 최적화
)

# 텍스트 생성 파이프라인 설정
gpt_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

# 사전 학습된 임베딩 모델 로드
embedding_model = SentenceTransformer('jhgan/ko-sroberta-multitask')

# ChromaDB 클라이언트 초기화
chroma_client = chromadb.Client(Settings(
    persist_directory="/content/drive/MyDrive/Colab Notebooks/chroma_db" # 이전에 설정한 경로와 동일
))

# 기존 컬렉션 가져오기
#collection_name = "ai_industry_trends"
collection_name = "Flight_Manual"
collection = chroma_client.get_collection(collection_name)

# 다양한 쿼리 표현을 생성하는 함수 (정비교범으로 테스트 시 변경 필요)
def generate_diverse_queries(query):
    variations = [
        query,
        f"How does {query} impact the industry?",
        f"Explain the trends in {query}",
        f"What are the latest updates in {query}?",
        f"Recent advancements in {query}",
    ]
    return variations

# 통합된 Retriever 클래스: 단일 쿼리 또는 다양한 쿼리로 검색 가능
class UnifiedQueryRetriever:
    def __init__(self, embedding_model, collection, use_diverse_queries=False):
        self.embedding_model = embedding_model
        self.collection = collection
        self.use_diverse_queries = use_diverse_queries

    def search(self, query, top_k=5):
        # 다양한 쿼리를 생성할지 여부에 따라 처리
        queries = generate_diverse_queries(query) if self.use_diverse_queries else [query]

        # 각 쿼리에 대해 임베딩 생성 및 검색 수행
        query_embeddings = self.embedding_model.encode(queries, convert_to_tensor=True)
        results = []
        for query_embedding in query_embeddings:
            search_results = self.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )
            # 유효한 결과만 수집
            if search_results and 'documents' in search_results:
                documents = search_results.get('documents', [[]])
                if documents:
                    for doc_list in documents:
                        if isinstance(doc_list, list):
                            results.extend(doc_list)
                        else:
                            results.append(doc_list)

        # 중복 제거 후 반환
        return list(set(results))

# 텍스트 조각을 벡터 임베딩과 함께 컬렉션에 추가하는 함수
def store_embeddings_in_chroma(text_chunks, embeddings):
    ids = [f"doc_{i}" for i in range(len(text_chunks))]
    collection.add(
        ids=ids,
        embeddings=[embedding.tolist() for embedding in embeddings],
        metadatas=[{"text": chunk} for chunk in text_chunks],
        documents=text_chunks
    )
    print(f"Added {len(text_chunks)} documents to the ChromaDB collection.")

# 검색된 결과를 기반으로 프롬프트 작성
def generate_prompt_from_results(query, search_results, max_context_length=500):
    if not search_results:
        return None

    context = []
    for result in search_results[:3]:  # 상위 3개의 검색 결과만 사용
        if isinstance(result, dict) and 'text' in result:
            context.append(result['text'])
        elif isinstance(result, str):
            context.append(result)

    context = "\n".join(context)
    if len(context) > max_context_length:
        context = context[:max_context_length] + "..."  # 문맥이 길면 자르기

    return context

# 검색된 결과를 기반으로 답변 생성
def generate_answer(query, search_results, max_new_tokens=50):
    context = generate_prompt_from_results(query, search_results)
    if not context:
        return "관련된 정보를 찾을 수 없습니다. 다른 질문을 시도해 보세요."

    # 실제 답변 생성
    prompt = f"{context}\n\n답변:"
    response = gpt_pipeline(prompt, max_new_tokens=max_new_tokens, num_return_sequences=1, do_sample=True, temperature=0.5)

    # 출력에서 질문과 문맥 부분을 제거하고 실제 답변만 반환
    answer = response[0]['generated_text'].strip().replace(prompt, "").strip()
    return answer

# 대화 상태 관리 (이전 대화를 포함하여 문맥 유지)
class ChatBot:
    def __init__(self, use_diverse_queries=False):
        self.retriever = UnifiedQueryRetriever(embedding_model, collection, use_diverse_queries)

    def get_response(self, user_input):
        results = self.retriever.search(user_input)
        return generate_answer(user_input, results)

    def run(self):
        print("챗봇과 대화를 시작하세요. 종료하려면 'exit'을 입력하세요.")
        while True:
            user_input = input("\n사용자: ")
            if user_input.lower() == "exit":
                print("대화를 종료합니다.")
                break

            response = self.get_response(user_input)
            print(f"\n챗봇:{response}")

# 챗봇 실행
chatbot = ChatBot(use_diverse_queries=True)
chatbot.run()


tokenizer_config.json:   0%|          | 0.00/51.8k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/709 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00006-of-00007.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00007-of-00007.safetensors:   0%|          | 0.00/2.57G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

챗봇과 대화를 시작하세요. 종료하려면 'exit'을 입력하세요.

사용자: CHANGE SYMBOL은 무엇을 의미하는거야?

챗봇:위의 지문에 근거하여 "전투기"라고 답할 수 있는 질문은 무엇인가요?

선택지: (1). 예 (2). 아니요
정답은 (1). 예입니다.
그 이유는

사용자: TA-50 항공기에 대해서 설명해줘

챗봇:IP를 사전준비함으로써, 조종사는 표적을 정하기 위해 공기 중에서 방위, 거리, 고저를 입력할 수 있다. VIP 조준 옵션의 수동 초기화(초
